In [6]:
import numpy as np
import random
import copy


class environement:
    
    def __init__(self):
        self.state=None
        self.width=None
        self.height=None
        self.snake_list=[]
        self.foodx=None
        self.foody=None
        self.distance=None
        self.lastmove=None
        self.score=0
        self.past_pos=None
        self.hunger=None
    def env_init(self,info={}):
        self.width=info.get("width",10)
        self.height=info.get("height",10)
        self.state=np.zeros((self.width,self.height))
    def spawnfood(self):
        foodx=random.randint(0,self.width-1)
        foody=random.randint(0,self.height-1)
        for x in self.snake_list:
            if(foodx==x[0] and foody==x[1]):
                foodx,foody=self.spawnfood()
        return foodx,foody
    def Distance(self,snake_head,foodx,foody):
        return (abs((snake_head[0]-foodx))+abs((snake_head[1]-foody)))
    def arrange_state(self):
        self.state=np.zeros((self.width,self.height))
        for x,y ,in self.snake_list:
            if(x==self.snake_list[-1][0] and y==self.snake_list[-1][1]):
                self.state[x][y]=2
            else:
                self.state[x][y]=1

        self.state[self.foodx][self.foody]=5

        
    def start(self):
        self.snake_head=[int(self.width/2),int(self.height/3)]
        self.snake_list=[]
        a=[]
        a.append(self.snake_head[0])
        a.append(self.snake_head[1])
        self.snake_list.append(a)
        self.foodx,self.foody=self.spawnfood()

        self.hunger=0
        self.arrange_state()
        self.distance=self.Distance(self.snake_head,self.foodx,self.foody)
        self.score=0
        self.past_pos= self.state
        state=self.mixstate()
        return state
    def rew(self,reward):
            if(reward=="food"):
                return 1
            elif(reward=="death"):
                return -1
            elif(reward=="moveclose"):
                return 0
            elif(reward=="movefar"):
                return 0
    def step(self,action):
        #action 0:L 1:R 2:D 3:U:
        self.hunger+=1
        action=self.canGo(action)
        self.lastmove=action
        self.past_pos=self.state.copy()
        return self.step_arrange(action)
    def step_arrange(self,action):
        if action==0:
            index=1
            incr=-1
            side=self.height
        elif action==1:
            index=1
            incr=1
            side=self.height
        elif action==2:
            index=0
            incr=-1
            side=self.width
        else :        
            index=0
            incr=1
            side=self.width
        self.snake_head[index]+=incr
        a=[]
        a.append(self.snake_head[0])
        a.append(self.snake_head[1])
        if(self.snake_head[0]==self.foodx and self.snake_head[1]==self.foody):
            self.snake_list.append(a)
            self.foodx,self.foody=self.spawnfood()
            self.arrange_state()
            self.score+=1
            state=self.mixstate()
            self.hunger=0
            return self.rew("food"),state,False,self.score
        if self.snake_head[index]<0 or self.snake_head[index]>=side or self.hunger==100:
            #self.start()
            state=self.mixstate()
            return self.rew("death"),state,True,self.score
        for x in self.snake_list[1:]:
            if x == self.snake_head:
                #self.start()
                state=self.mixstate()
                return self.rew("death"),state,True,self.score
        del self.snake_list[0]
        self.snake_list.append(a)
        self.arrange_state()
        new_distance=self.Distance(self.snake_head,self.foodx,self.foody)
        if new_distance>=self.distance :
            self.distance=new_distance
            state=self.mixstate()
            return self.rew("movefar"),state,False,self.score
        else:
            self.distance=new_distance
            state=self.mixstate()
            return self.rew("moveclose"),state,False,self.score
    def canGo(self,action):
        if(self.lastmove==0 and action==1):
            return self.lastmove
        elif(self.lastmove==1 and action==0):
            return self.lastmove
        elif(self.lastmove==2 and action==3):
            return self.lastmove
        elif(self.lastmove==3 and action==2):
            return self.lastmove
        else :
            return action
    def mixstate(self):
        state=np.array([self.state,self.past_pos])
        state=np.rollaxis(state,2)
        state=np.rollaxis(state,2)
        return state
        
        
        
        
   
        
        

In [7]:
env=environement()

In [8]:
env.env_init({})

In [9]:
env.start()




array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 3., 5., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [5]:
env.step(1)


(0.1,
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 5., 0., 0.]]),
 False,
 0)